# INTELIGENCIA ARTIFICIAL

## Proyecto 1: Clasificación y Regresión Lineal

Hecho por: Juan Sebastián Clavijo Martínez<br />
TEMA: Usar el dataset sobre dígitos manuscritos (MNIST) para implementar clasificacion lineal y datos artificiales suministrados por el profesor para modelar la relación entre variables mediante regresión lineal<br />
Fecha: 10-09-2024<br />
Notas: Nada por ahora<br />
**Pontificia Universidad Javeriana**


In [ ]:
# Importado de bibliotecas pertinentes
# se usa el #type: ignore para que mi vscode genere warnings sobre la resolucion de dependencias de los pkts

import pandas as pd  # type: ignore # tratamiento de datos en un dataframe
import numpy as np  # type: ignore # numerical python
import seaborn as sns  # type: ignore # biblioteca versatil para estadistica y visualizacion
import matplotlib.pyplot as plt  # type: ignore # biblioteca para vizualizacion de datos
from sklearn.datasets import load_digits # repo donde estan las imagenes pertinentes para la primera parte del proyecto

### PROBLEMA 1: CLASIFICACIÓN  LINEAL

#### Parte 1.1 Procesamiento de datos

In [ ]:
# Parte 1.1.1: carga de datos
# se trae el dataset
digits = load_digits() #se cargan los datos con la funcion load_digits
print(digits.data.shape)
# Se imprime la forma de los datos para verificar el número de instancias y atributos.
# Según la documentación de scikit-learn (https://scikit-learn.org/stable/datasets/toy_dataset.html#digits-dataset),
# este dataset contiene 1797 instancias, cada una con 64 atributos, que corresponden a una imagen de 8x8 píxeles.
# según la documentacion, este dataset no tiene datos NaN o NULL, por eso no se realiza el paso 1.1.2 limpieza y preparacion de los datos

In [ ]:
# Parte 1.1.2 visualización de todos los dígitos organizados
# Configurar la cuadrícula 2x5
fig, axes = plt.subplots(2, 5, figsize=(10, 7))
plt.gray()

# Mostrar cada dígito en la cuadrícula
for i, ax in enumerate(axes.flat):
    ax.matshow(digits.images[i])
    ax.set_title(f"Dígito: {digits.target[i]}")
    ax.axis("off")  #se ocultan los ejes para una visualización más limpia

plt.show()

### Análisis descriptivo de las imagenes

- son bonitas
- algo mas


## Parte 1.2 Implementación de ?

In [ ]:
# Parte 1.2.1: implementacion del código

# Aquí debería ir código
print("código")

### PROBLEMA 2: Regresión Lineal

#### parte 2.1 procesamiento de datos

In [ ]:
# Parte 2.1.1: carga de datos
# se trae el dataset
# Aunque nos proporcionaron el archivo.csv y viene en la carpeta de este proyecto, decidí colgarlo en github para más
# comodidad y por si algo le pasa, que pueda ser facilmente traído a través de internet
url = "https://raw.githubusercontent.com/jclavijomartinez/IA2430/master/Proyecto1/RegressionData.csv" 
# se hace el obj dataframe
df_artif = pd.read_csv(url) # Se llama asi porque la base de datos es artificial
# se presentan los primeros 5 renglones
df_artif.head(5)

In [ ]:
# Parte 2.1.2: limpieza y preparación de los datos (cuenta de datos null, NaN, etc. y limpieza)
desaparecidos = len(df_artif) - len(df_artif.dropna())
Cantidad = len(df_artif)
print("Cantidad de datos observados con datos NaN", desaparecidos)
print("Cantidad de datos duplicados", df_artif.duplicated().sum())
##se eliminan los datos Null y Duplicados
df_artif.dropna(inplace=True)
df_artif.drop_duplicates(inplace=True)
# Se reinicia el indice por los elementos eliminados
df_artif.reset_index(drop=True, inplace=True)